In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
import queue
import math
from enum import Enum
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

In [290]:
data_path = 'D:\\Projects\\MCTS\\datasets\\'
file_names = [{'name':'blood-transfusion-service-center.csv', 'target_class': 'Class', 'pos_class': 'numerical'},
             {'name':'credit-g.csv', 'target_class': 'class', 'pos_class': 'good'},
             {'name':'kr-vs-kp.csv', 'target_class': 'class', 'pos_class': '\'won\''},
             {'name':'monks-problems-2.csv', 'target_class': 'class', 'pos_class': 'numerical'},
             {'name':'diabetes.csv', 'target_class': 'class', 'pos_class': 'tested_positive'},
             {'name':'qsar-biodeg.csv', 'target_class': 'Class', 'pos_class': 'numerical'},
             {'name':'steel-plates-fault.csv', 'target_class': 'Class', 'pos_class': 'numerical'},
             {'name':'tic-tac-toe.csv', 'target_class': 'Class', 'pos_class': 'positive'},
             {'name':'wdbc.csv', 'target_class': 'Class', 'pos_class': 'numerical'},
             {'name':'hill-valley.csv', 'target_class': 'Class', 'pos_class': 'numerical'},
             {'name':'pc1.csv', 'target_class': 'defects', 'pos_class': 'true'},
             {'name':'spambase.csv', 'target_class': 'class', 'pos_class': 'numerical'},
             {'name':'artificial.csv', 'target_class': 'class', 'pos_class': 'numerical'}]

## Metrics class

In [3]:
class BuildInMetrics:
    def __init__(self):
        self._metrics = {
            'acc': accuracy_score,
            'f1': f1_score,
            'roc_auc': roc_auc_score
        }
        
    def get_metric(self, name):
        if name in self._metrics:
            return self._metrics[name]
        else:
            print('Error initializing MCTS object, \"' + name + '\" is not supported metric, available values are: ' + ', '.join(self._metrics.keys()))
            return None

## Scoring function

In [233]:
class ScoringFunctions():
    def __init__(self, name, params):
        self._params = params
        self._name = name
    
    def set_scoring_function(self, name, params = None):
        self._name = name
        if(params is not None):
            self._params = params 
    
    def get_score(self, node, scores):
        if(self._name == 'default'):
            return self._default_scoring(node)
        elif(self._name == 'with_variance'):
            return self._var_scoring(node)
        elif(self._name == 'g_rave'):
            return self._g_rave_scoring(node, scores)
        else:
            print('Error initializing MCTS object, \"' + + '\"')
            return None
    
    def _default_scoring(self, node):
        if(node._parent_node == None or node.T == 0):
            return float("Inf")
        else:
            return node.get_score() + math.sqrt(self._params['c_e'] * math.log(node._parent_node.T)/node.T)
        
    def _var_scoring(self, node):
        if(node._parent_node == None or node.T == 0):
            return float("Inf")
        else:
            return node.get_score() + math.sqrt(self._params['c_e'] * math.log(node._parent_node.T)/node.T)
    
    def _g_rave_scoring(self, node, scores):
        g_scores = scores['g_rave']
        
        if(node._parent_node == None or node.T == 0):
            if(node.feature_name not in g_scores):
                return float("Inf")
            else:
                return g_scores[node.feature_name]['score']
        else:
            c = self._params['c']
            c_l = self._params['c_l']
            alpha = c/(c + node.T)
            beta = c_l/(c_l + node.T)
            
            return ((1 - alpha) * node.get_score() + 
                alpha * g_scores[node.feature_name]['score'] + 
                math.sqrt(self._params['c_e'] * math.log(node._parent_node.T)/node.T) *
                min(0.25, node.get_variance() + math.sqrt(2 * math.log(node._parent_node.T)/node.T)))

In [206]:
sf = ScoringFunctions('g_rave', DefaultSettings.get_default_params())
gs = GlobalScores()
root = Node('')
root.add_child_nodes(['a','b','c','d'])
root.child_nodes[0].update_scores_up(1, gs)
root.child_nodes[1].update_scores_up(0.8, gs)
root.child_nodes[2].update_scores_up(0.6, gs)
root.child_nodes[3].update_scores_up(0.4, gs)
root.child_nodes[0].add_child_nodes(['b','c','d'])

In [207]:
root.child_nodes[0].child_nodes[0].update_scores_up(0.8,gs)
root.child_nodes[0].child_nodes[0].update_scores_up(0.8,gs)
root.child_nodes[0].child_nodes[0].update_scores_up(0.8,gs)
root.child_nodes[0].child_nodes[0].update_scores_up(0.8,gs)
root.child_nodes[0].child_nodes[0].update_scores_up(0.8,gs)
root.child_nodes[0].child_nodes[0].update_scores_up(0.8,gs)
root.child_nodes[0].child_nodes[0].update_scores_up(0.8,gs)
root.child_nodes[0].child_nodes[0].update_scores_up(0.8,gs)
root.child_nodes[0].child_nodes[0].update_scores_up(0.8,gs)
root.child_nodes[0].child_nodes[0].update_scores_up(0.8,gs)
root.child_nodes[0].child_nodes[0].update_scores_up(0.8,gs)
root.child_nodes[0].child_nodes[0].update_scores_up(0.8,gs)
root.child_nodes[0].child_nodes[0].update_scores_up(0.8,gs)
root.child_nodes[0].child_nodes[0].update_scores_up(0.8,gs)
root.child_nodes[0].child_nodes[0].update_scores_up(0.8,gs)
root.child_nodes[1].update_scores_up(0.7,gs)

In [213]:
sf.get_score(root.child_nodes[1], gs.scores)

0.3333333333333333
0.75
0.7941176470588236
1.7308183826022854


1.1974104780035124

## Global scores

In [98]:
class GlobalScores:
    def __init__(self):
        self.scores = {'g_rave': {}}
    
    def update_g_rave_score(self, name, score):
        if(name not in self.scores['g_rave']):
            self.scores['g_rave'][name] = {'n': 1, 'score': score}
        else:
            n = self.scores['g_rave'][name]['n']
            t_score = (self.scores['g_rave'][name]['score'] * n + score)/(n + 1)
            self.scores['g_rave'][name] = {'n': n + 1, 'score': t_score}

In [ ]:
gs = GlobalScores()
gs.

## Node class

In [222]:
class Node:
    def __init__(self, feature_name, parent_node = None, is_subtree_full = False):
        self.feature_name = feature_name
        self.child_nodes = []
        self.T = 0
        self.score_sum = 0
        self._is_subtree_full = is_subtree_full
        self._parent_node = parent_node
        self._scores = []
        
    def add_child_node(self, node_name, is_subtree_full = False):
        new_node = Node(node_name, self, is_subtree_full)
        self.child_nodes.append(new_node)
        
    def add_child_nodes(self, node_names):
        for name in node_names:
            self.add_child_node(name, len(node_names) == 1)
      
    def update_node(self, score, scores):
        self.score_sum += score
        self.T += 1
        self._scores.append(score)
        scores.update_g_rave_score(self.feature_name, score)
        if(len(self.child_nodes) != 0):
            is_subtree_full = True
            for node in self.child_nodes:
                if(not node._is_subtree_full):
                    is_subtree_full = False
                    break
            self._is_subtree_full = is_subtree_full
    
    def get_score(self):
        return float('Inf') if self.T == 0 else self.score_sum/self.T
    
    def get_variance(self):
        return np.var(self._scores)
    
    def update_scores_up(self, score, scores):
        current_node = self
        
        while(current_node != None):
            current_node.update_node(score, scores)
            current_node = current_node._parent_node

In [6]:
# Scenario 1: Creating a Node
print("Scenario 1:")
root = Node("")
print(root.feature_name == "")
print(root.T == 0)
print(root._is_subtree_full == False)
print(root._parent_node == None)
print(root.T == 0)
print(root.get_score() == float('Inf'))

# Scenario 2: Adding child nodes
print("Scenario 2:")
root.add_child_nodes(['b','c','d'])
print(len(root.child_nodes) == 3)
print(root.child_nodes[0].feature_name == 'b')
print(root.child_nodes[1].feature_name == 'c')
print(root.child_nodes[2].feature_name == 'd')
root.child_nodes[0].update_scores_up(1)
print(root.child_nodes[0].T == 1)
print(root.child_nodes[0].get_score() == 1)
print(root.get_score() == 1)
print(root.T == 1)
root.child_nodes[1].update_scores_up(0.8)
print(root.child_nodes[1].T == 1)
print(root.child_nodes[1].get_score() == 0.8)
print(root.get_score() == 0.9)
print(root.T == 2)
root.child_nodes[2].update_scores_up(0.6)
print(root.child_nodes[2].T == 1)
print(root.child_nodes[2].get_score() == 0.6)
print(root.get_score() == 2.4/3)
print(root.T == 3)

# Another level
print("Scenario 3")
root.child_nodes[0].add_child_nodes(['c','d'])
print(len(root.child_nodes[0].child_nodes) == 2)
print(root.child_nodes[0].child_nodes[0].feature_name == 'c')
print(root.child_nodes[0].child_nodes[1].feature_name == 'd')
print(root.child_nodes[0].child_nodes[0]._is_subtree_full == False)
root.child_nodes[0].child_nodes[0].update_scores_up(0.2)
print(root.child_nodes[0].child_nodes[0].T == 1)
print(root.child_nodes[0].child_nodes[0].get_score() == 0.2)
print(root.child_nodes[0].T == 2)
print(root.child_nodes[0].get_score() == 0.6)
print(root.T == 4)
print(root.get_score() == 2.6/4)
# Scenario 3: last 
print("Scenario 4:")
root.child_nodes[0].child_nodes[0].add_child_nodes(['d'])
print(len(root.child_nodes[0].child_nodes[0].child_nodes) == 1)
print(root.child_nodes[0].child_nodes[0].child_nodes[0]._is_subtree_full)
print(root.child_nodes[0].child_nodes[0]._is_subtree_full == False)
root.child_nodes[0].child_nodes[0].child_nodes[0].update_scores_up(0.4)
print(root.child_nodes[0].child_nodes[0]._is_subtree_full)
print(root.child_nodes[0].child_nodes[1]._is_subtree_full == False)
print(root.child_nodes[0]._is_subtree_full == False)
print(root.child_nodes[0].child_nodes[0].child_nodes[0].T == 1)
print(root.child_nodes[0].child_nodes[0].child_nodes[0].get_score() == 0.4)
print(root.child_nodes[0].child_nodes[0].T == 2)

Scenario 1:
True
True
True
True
True
True
Scenario 2:
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
Scenario 3
True
True
True
True
True
True
True
True
True
True
Scenario 4:
True
True
True
True
True
True
True
True
True


## Multiarm strategies

In [283]:
class MultiArmStrategies:
    def __init__(self, name, all_node_names):
        self._name = name
        self._all_node_names = all_node_names
        
    def multiarm_strategy(self, node, used_features, scoring_function, other_scores):
        if(self._name == 'default'):
            return self._default_strategy(node, used_features, scoring_function, other_scores)
        else:
            return None    
    
    def _default_strategy(self, node, used_features, scoring_function, other_scores):
        #print("default strategy: " + node.feature_name)
        if(len(node.child_nodes) == 0):
            #print("first if")
            self._add_child_nodes(node, used_features)
            return node.child_nodes[0]
        else:
            #print("else")
            best_score = 0
            best_node = None
            tmp_score = 0
            
            for child_node in node.child_nodes:
                if(not child_node._is_subtree_full):
                    score = scoring_function(child_node, other_scores)
                    if(score > best_score):
                        best_score = score
                        best_node = child_node
            return best_node
  
    def _add_child_nodes(self, node, used_features):
        #print('adding nodes to ' + node.feature_name + ' :' + ' '.join(self._all_node_names - used_features))
        node.add_child_nodes(self._all_node_names - used_features)   

## EndStrategies

In [8]:
class EndStrategies:
    def __init__(self, name):
        self._name = name
    
    def are_calculations_over(self, node, params):
        if(self._name == 'default'):
            return self._first_new_strategy(node, params)
        else:
            return None
        
    def _first_new_strategy(self, node, params):
        #print("_first_new_strategy:")
        if(node.T > 0 and not node._is_subtree_full):
            #print("first if")
            return False
        else:
            if(node._parent_node == None):
                #print("second if, no parent node")
                return False
            else:
                #print("else")
                return True


## DefaultSettings

In [59]:
class DefaultSettings:
    @staticmethod
    def get_default_params():
        return {
            "c_e": 2,
            "c": 1,
            "c_l": 1
        }
    

## MCTS

In [286]:
class MCTS:
    def __init__(self, 
                 model,
                 task = 'classification',
                 calculactions_done_conditions = {'type': 'iterations', 'max_val': 10},
                 params = None,
                 metric = 'acc', 
                 scoring_function = 'g_rave', 
                 multiarm_strategy = 'default', 
                 end_strategy = 'default'):
        self._metric_name = metric
        self._scoring_function_name = scoring_function
        self._multiarm_strategy_name = multiarm_strategy
        self._end_strategy_name = end_strategy
        self._best_features = None
        self._best_score = 0
        self._task = task
        self._root = Node("")
        self._feature_names = None
        self._calculactions_done_conditions = calculactions_done_conditions
        self._model = model
        self._time = 0
        self._iterations = 0
        
        if(params is None):
            self._params = DefaultSettings.get_default_params()
        
    def fit(self, data, out_variable):
        
        if(self._task == 'classification'):
            self._classification_fit(data, out_variable)
        else:
            self._regression_fit(data, out_variable)
            
    def _classification_fit(self, data, out_variable):
        self._init_fitting_values(data)
        while not self._is_fitting_over():
            self._single_classification_iteration(data, out_variable)
        
        self._model.fit(data.loc[:, self._best_features], out_variable)
    
    def _regression_fit(self, data, out_variable):
        return None
    
    def _single_classification_iteration(self, data, out_variable):
        #print('classification iteration')
        used_features = set()
        node = self._root
        is_iteration_over = False
        while not is_iteration_over:
            node = self._multiarm_strategy.multiarm_strategy(node, used_features, self._scoring_function.get_score, self._global_scores.scores)
            #print("selected feature: " + node.feature_name)
            is_iteration_over = self._end_strategy.are_calculations_over(node, self._params)
            used_features.add(node.feature_name)
            #print("--------")
        
        X_train, X_test, y_train, y_test = train_test_split(data.loc[:,used_features], out_variable, test_size = 0.25, random_state = 123)
        self._model.fit(X_train, y_train)
        predicted = self._model.predict(X_test)
        score = self._metric(y_test, predicted)
        #print('score: ' + str(score))
        node.update_scores_up(score, self._global_scores)
        #print(used_features)
        if(score > self._best_score):
            self._best_score = score
            self._best_features = used_features
        
        if(self._longest_tree_branch < len(used_features)):
            self._longest_tree_branch = len(used_features)
        
        #print("----------END OF ITERATION----------")
    
    def _single_regression_iteration(self):
        return None
    
    def _init_fitting_values(self, data):
        self._feature_names = set(data.columns)
        self._multiarm_strategy = MultiArmStrategies(self._multiarm_strategy_name, self._feature_names)
        self._end_strategy = EndStrategies(self._end_strategy_name)
        self._scoring_function = ScoringFunctions(self._scoring_function_name, self._params)
        self._metric = BuildInMetrics().get_metric(self._metric_name)
        self._best_features = None
        self._best_score = 0
        self._iterations = 0
        self._longest_tree_branch = 0
        self._time = time.time()
        self._global_scores = GlobalScores()
    
    def _is_fitting_over(self):
        if(self._root._is_subtree_full):
            print('Whole tree searched, finishing prematurely')
            return True
        
        if(self._calculactions_done_conditions['type'] == 'iterations'):
            self._iterations += 1
            return self._iterations > self._calculactions_done_conditions['max_val'] 
        else:
            #print('Time ellapsed: ' + str(time.time() - self._time))
            return (time.time() - self._time) > self._calculactions_done_conditions['max_val'] 
        
    def predict(self, data):
        return self._model.predict(data.loc[:, self._best_features])
    
    def predict_proba(self, data):
        return self._model.predict_proba(data.loc[:, self._best_features])

In [229]:
data = pd.read_csv(data_path + file_names[6]['name'])
data_labels = data.loc[:,file_names[6]['target_class']]
data = data.drop(columns = [file_names[6]['target_class']])
X_train, X_test, y_train, y_test = train_test_split(data, data_labels, test_size = 0.25, random_state = 123)

In [295]:
data = pd.read_csv(data_path + file_names[12]['name'])
labels = data['class']
labels[labels == -1] = 0
data = data.drop(columns = ['class'])

In [297]:
kf = KFold(n_splits = 5, random_state = 123, shuffle=True)
model = LogisticRegression(solver='liblinear')
auc_score = 0
acc_score = 0
f1_score_ = 0
for train, test in kf.split(data):
    model.fit(data.loc[train,:], labels[train])
    predicted = model.predict(data.loc[test,:])
    p_proba = model.predict_proba(data.loc[test,:])[:,1]
    auc_score += roc_auc_score(labels[test] == 1, p_proba)
    acc_score += accuracy_score(labels[test], predicted)
    f1_score_ += f1_score(labels[test], predicted, pos_label = 1)

print('auc: ' + str(auc_score/5))
print('acc: ' + str(acc_score/5))
print('f1: ' + str(f1_score_/5))

auc: 0.5375857219608258
acc: 0.5325
f1: 0.5348084947997962


In [300]:
model = LogisticRegression(solver='liblinear')
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.25, random_state = 123)
mcts = MCTS(model, calculactions_done_conditions = {'type': 'time', 'max_val': 600}, metric='acc')
mcts.fit(X_train, y_train)

In [301]:
predicted = mcts.predict(X_test)
p_proba = mcts.predict_proba(X_test)
print(roc_auc_score(y_test == 1, p_proba[:,1]))
print(accuracy_score(y_test, predicted))
print(f1_score(y_test, predicted, pos_label = 1))

0.6199731045081968
0.59
0.5841784989858011


In [302]:
mcts._best_features

{'x103',
 'x111',
 'x113',
 'x115',
 'x116',
 'x121',
 'x125',
 'x127',
 'x130',
 'x133',
 'x140',
 'x141',
 'x142',
 'x147',
 'x156',
 'x161',
 'x182',
 'x186',
 'x19',
 'x192',
 'x197',
 'x198',
 'x201',
 'x206',
 'x209',
 'x211',
 'x216',
 'x220',
 'x221',
 'x225',
 'x232',
 'x239',
 'x244',
 'x248',
 'x25',
 'x250',
 'x260',
 'x262',
 'x265',
 'x269',
 'x275',
 'x281',
 'x289',
 'x30',
 'x300',
 'x307',
 'x318',
 'x32',
 'x324',
 'x326',
 'x33',
 'x345',
 'x353',
 'x358',
 'x364',
 'x365',
 'x367',
 'x370',
 'x378',
 'x379',
 'x381',
 'x382',
 'x391',
 'x403',
 'x410',
 'x412',
 'x421',
 'x425',
 'x427',
 'x429',
 'x43',
 'x432',
 'x435',
 'x436',
 'x440',
 'x441',
 'x442',
 'x445',
 'x446',
 'x45',
 'x453',
 'x454',
 'x455',
 'x46',
 'x464',
 'x466',
 'x469',
 'x476',
 'x477',
 'x480',
 'x491',
 'x493',
 'x494',
 'x496',
 'x497',
 'x498',
 'x50',
 'x500',
 'x56',
 'x62',
 'x64',
 'x76',
 'x86',
 'x91',
 'x94',
 'x99'}